In [ ]:
import hoomd
import numpy as np
# import ex_render

from monk import mimse, pair, prep

In [ ]:
seed = 1000
cpu = hoomd.device.CPU()
sim = hoomd.Simulation(cpu, seed=seed)
N = 512
rng = prep.init_rng(seed + 1)
L = prep.len_from_phi(N, 1.2, dim=3)
snap = prep.approx_euclidean_snapshot(N, L, rng, dim=3, ratios=[80,20], diams=[1.0, 0.88])

sim.create_state_from_snapshot(snap)
# sim.create_state_from_gsd()

tree = hoomd.md.nlist.Tree(0.2)

integrator = hoomd.md.Integrator(dt=0.0025)
lang = hoomd.md.methods.Langevin(hoomd.filter.All(), 1.5)
lj = pair.KA_LJ(tree)
integrator.forces = [lj]
integrator.methods = [lang]
sim.operations.integrator = integrator

sim.run(0)
sim.state.thermalize_particle_momenta(hoomd.filter.All(), 1.5)

sim.run(1000)

sim.operations.integrator = None

integrator.forces.pop()
del integrator, lang

In [ ]:
hoomd.write.GSD.write(sim.state, "test-3d.gsd")

In [ ]:
fire = hoomd.md.minimize.FIRE(0.0025,
                              force_tol=1e-2,
                              angmom_tol=1e-2,
                              energy_tol=1e-7)

mimse_force = mimse.MIMSEForce(sim.state.box)
nve = hoomd.md.methods.NVE(hoomd.filter.All())
fire.forces = [lj]
fire.methods = [nve]

sim.operations.integrator = fire

In [ ]:
sim.operations.writers.clear()

In [ ]:
thermodynamic_properties = hoomd.md.compute.ThermodynamicQuantities(
    filter=hoomd.filter.All())
sim.operations.computes.append(thermodynamic_properties)

logger = hoomd.logging.Logger(categories=["scalar"])

logger.add(thermodynamic_properties, ["potential_energy"])

writer = hoomd.write.GSD(hoomd.trigger.Periodic(40), filename='test_mimse-3d.gsd', mode="wb", log=logger)

sim.operations.writers.append(writer)

table = hoomd.write.Table(trigger=hoomd.trigger.Periodic(period=40),
                          logger=logger)

# sim.operations.writers.append(table)

In [ ]:
sim.run(8000)

In [ ]:
fire.forces.append(mimse_force)

In [ ]:
for i in range(50):
    fire.reset()
    while not fire.converged:
        sim.run(40)
    print(lj.energy, mimse_force.energy)
    snap = sim.state.get_snapshot()
    mimse_force.add_bias(20.0, 1.0, snap.particles.position, np.zeros_like(snap.particles.position))

In [ ]:
mimse_force.nn

In [ ]:
lj.energy

In [ ]:
len(mimse_force.nn)

In [ ]:
sim.run(0)

In [ ]:
x = hoomd._hoomd.make_scalar3(1,10,0)
im = hoomd._hoomd.make_int3(0,0,1)
flags = hoomd._hoomd.make_char3(0,0,0)
y = sim.state.box._cpp_obj.wrap(x, im, flags)